In [3]:
import pandas as pd
df = pd.read_csv("../data/processed/risk_dataset.csv")
df.columns = ['_', 'doc_id', 'date', 'edinet_code', 'sec_code', 'filer_name', 'risk_text']

In [6]:
import os, sys

# 📌 現在 /app/notebooks なので、../ で /app に戻る
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.insert(0, os.getcwd())
from src.preprocessing import text_preprocessor

df["cleaned_risk_text"] = df["risk_text"].fillna("").apply(text_preprocessor.clean_text)

# 事業リスクテキストをトークン化（品詞・ストップワード考慮済）
df["tokens"] = df["cleaned_risk_text"].apply(text_preprocessor.tokenize_with_filters)

# 空白区切りのトークン文字列（例：Word2VecやBERT入力用などに便利）
df["token_str"] = df["tokens"].apply(lambda x: " ".join(x))

In [7]:
df['token_str']

0       ４ 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 投...
1       ４ 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 投...
2       ４ 事業 リスク 当社 グループ 事業 関し リスク 要因 なる 考え 事項 以下 あり 文...
3       ４ 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 投...
4       ４ 事業 リスク 当社 グループ 定期的 リスクアセスメント 実施 し リスク 洗い出し 評...
                              ...                        
2188    3 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 経...
2189    3 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 経...
2190    ３ 事業 リスク 有価証券 報告書 記載 し 事業 状況 経理 状況 関する 事項 うち 投...
2191    ３ 事業 リスク 1 方針 当社 大規模 災害 顧客 情報漏洩 予期 せ リスク 想定 し ...
2192    ３ 事業 リスク 当社 グループ 事業 展開 他 関する リスク 要因 なる 可能性 考え ...
Name: token_str, Length: 2193, dtype: object

In [10]:
df.to_csv("/data/processed/cleaned_risk_dataset.csv")

OSError: Cannot save file into a non-existent directory: '/data/processed'

## リスク文のベクトル化

In [17]:
import os
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# ---- 設定 ----
input_path = "data/processed/cleaned_risk_dataset.csv"   # 前処理済みの事業リスクデータ
output_path = "data/embeddings/bge_embeddings.npy"        # ベクトル保存先
model_name = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"

# ---- モデルロード ----
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = SentenceTransformer(model_name, device=device)

# ---- データ読み込み ----
df = pd.read_csv(input_path)

# テキスト列名を確認（適宜変更）
if "token_str" in df.columns:
    texts = df["token_str"].astype(str).tolist()
elif "cleaned_risk_text" in df.columns:
    texts = df["cleaned_risk_text"].astype(str).tolist()
else:
    raise ValueError("トークン化されたテキスト列（token_str など）が見つかりません")

# ---- ベクトル化 ----
print("Encoding texts...")

# 大量にある場合はバッチ処理（例：32件ずつ）
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # 類似度計算向け
)

# ---- 保存（NumPy形式）----
os.makedirs(os.path.dirname(output_path), exist_ok=True)
np.save(output_path, embeddings)
print(f"✅ Embeddings saved to {output_path}")

Using device: cuda
Encoding texts...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████| 69/69 [00:18<00:00,  3.71it/s]

✅ Embeddings saved to data/embeddings/bge_embeddings.npy


In [2]:
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer, VectorStoreIndex
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever

from langchain_community.llms.llamacpp import LlamaCpp
import faiss
import pickle

# --- LLMとエンベディングの設定 ---
print("Setting LLM and embedder...")
llm = LlamaCpp(model_path="../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf", temperature=0, n_ctx=1024)
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

# --- データ読み込み ---
print("Loading data...")
df = pd.read_csv("../data/processed/cleaned_risk_dataset.csv")
assert "token_str" in df.columns, "token_str カラムが見つかりません"
documents = [Document(text=row["token_str"]) for _, row in df.iterrows()]

# --- インデックス作成 ---
print("Creating FAISS index...")
dimension = 1024
faiss_index = faiss.IndexFlatL2(dimension)
faiss_store = FaissVectorStore(faiss_index=faiss_index)

index = VectorStoreIndex.from_documents(
    documents,
    vector_store=faiss_store,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=20)],
    show_progress=True,
)

# --- 保存 ---
with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)
print("✅ FAISS index saved to faiss_index.pkl")

# --- インデックスの読み込み ---
with open("faiss_index.pkl", "rb") as f:
    loaded_index = pickle.load(f)

# --- 検索セットアップ ---
retriever = VectorIndexRetriever(index=loaded_index, similarity_top_k=3)
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ../models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Llama-3-8B-optimal-merged-stage2
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              lla

Setting LLM and embedder...


llm_load_vocab: control token: 128255 '<|reserved_special_token_250|>' is not marked as EOG
llm_load_vocab: control token: 128254 '<|reserved_special_token_249|>' is not marked as EOG
llm_load_vocab: control token: 128253 '<|reserved_special_token_248|>' is not marked as EOG
llm_load_vocab: control token: 128251 '<|reserved_special_token_246|>' is not marked as EOG
llm_load_vocab: control token: 128246 '<|reserved_special_token_241|>' is not marked as EOG
llm_load_vocab: control token: 128243 '<|reserved_special_token_238|>' is not marked as EOG
llm_load_vocab: control token: 128240 '<|reserved_special_token_235|>' is not marked as EOG
llm_load_vocab: control token: 128239 '<|reserved_special_token_234|>' is not marked as EOG
llm_load_vocab: control token: 128238 '<|reserved_special_token_233|>' is not marked as EOG
llm_load_vocab: control token: 128237 '<|reserved_special_token_232|>' is not marked as EOG
llm_load_vocab: control token: 128232 '<|reserved_special_token_227|>' is not ma

Loading data...
Creating FAISS index...


Parsing nodes:   0%|          | 0/2193 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# テスト
# --- クエリ例 ---
question = "為替変動によるリスクについて教えてください。"
print(f"Q: {question}")
response = query_engine.query(question)
print(f"A: {response}")